In [3]:
# Clone Ultralytics (only if not done before)
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .

!pip install ultralytics --upgrade -q
from ultralytics import YOLO
import os


fatal: destination path 'ultralytics' already exists and is not an empty directory.
/content/ultralytics
Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.132-0.editable-py3-none-any.whl size=24148 sha256=be37d4777021bff777bff5897ff02c9f7083c8cb151ea765ad9567ee22b082d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-gs_5soss/wheels/ea/71/6b/a9012dfb148489fd8125c2310e565414c996b3c7721defe799
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.132
    Uninstalling ultralytics-8.3.132:
      Successfully uninstalled ultralytics-8.3.132


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
DATA_DIR = "/content/drive/MyDrive/PlantDec"
LOCAL_DATA_DIR = "/content/PlantDec"

import shutil
if not os.path.exists(LOCAL_DATA_DIR):
    shutil.copytree(DATA_DIR, LOCAL_DATA_DIR)

DATA_DIR = LOCAL_DATA_DIR  # Use fast local path


In [6]:
from collections import Counter
from glob import glob

def analyze_class_distribution(label_dir):
    counter = Counter()
    for label_file in glob(f"{label_dir}/*.txt"):
        with open(label_file, "r") as f:
            for line in f:
                class_id = int(line.strip().split()[0])
                counter[class_id] += 1
    return counter

train_labels = f"{DATA_DIR}/labels/train"
dist = analyze_class_distribution(train_labels)

# Print class counts
import yaml
with open(os.path.join(DATA_DIR, "data.yaml"), "r") as f:
    class_names = yaml.safe_load(f)["names"]

print("\n📊 Class Distribution:")
for i, count in dist.items():
    print(f"{i} ({class_names[i]}): {count} samples")

max_count = max(dist.values())



📊 Class Distribution:
29 (grape leaf): 1947 samples
0 (Apple Scab Leaf): 9 samples
14 (Raspberry leaf): 1 samples
24 (Tomato leaf yellow virus): 1 samples
12 (Potato leaf late blight): 1 samples
22 (Tomato leaf late blight): 1 samples
4 (Bell_pepper leaf): 1 samples
25 (Tomato leaf): 1 samples
7 (Corn Gray leaf spot): 1 samples
5 (Blueberry leaf): 1 samples
3 (Bell_pepper leaf spot): 1 samples
1 (Apple leaf): 1 samples
18 (Strawberry leaf): 1 samples
20 (Tomato Septoria leaf spot): 1 samples
9 (Corn rust leaf): 1 samples
23 (Tomato leaf mosaic virus): 1 samples
8 (Corn leaf blight): 1 samples
2 (Apple rust leaf): 1 samples
21 (Tomato leaf bacterial spot): 1 samples
27 (Tomato two spotted spider mites leaf): 1 samples
19 (Tomato Early blight leaf): 1 samples
28 (grape leaf black rot): 1 samples
17 (Squash Powdery mildew leaf): 1 samples
16 (Soybean leaf): 1 samples
15 (Soyabean leaf): 1 samples


In [7]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import random

def augment_image(img):
    img = Image.fromarray(img)
    if random.random() < 0.5:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)
    if random.random() < 0.5:
        img = img.transpose(Image.FLIP_TOP_BOTTOM)
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(random.uniform(0.8, 1.2))
    return np.array(img)

AUG_PATH = f"{DATA_DIR}/images/train"
LABEL_PATH = f"{DATA_DIR}/labels/train"

for class_id, count in dist.items():
    augment_needed = max_count - count
    if augment_needed <= 0:
        continue
    print(f"🧪 Augmenting class {class_id} ({class_names[class_id]}) with {augment_needed} samples")

    # Find all images containing this class
    for label_file in glob(f"{LABEL_PATH}/*.txt"):
        with open(label_file, "r") as f:
            lines = f.readlines()
        if any(int(line.split()[0]) == class_id for line in lines):
            base_name = os.path.basename(label_file).replace(".txt", "")
            img_path = os.path.join(AUG_PATH, base_name + ".jpg")
            if not os.path.exists(img_path):
                continue
            img = cv2.imread(img_path)
            for i in range(augment_needed // count + 1):
                aug_img = augment_image(img)
                new_img_name = f"{base_name}_aug_{i}.jpg"
                new_label_name = f"{base_name}_aug_{i}.txt"
                cv2.imwrite(os.path.join(AUG_PATH, new_img_name), aug_img)
                shutil.copyfile(label_file, os.path.join(LABEL_PATH, new_label_name))


🧪 Augmenting class 0 (Apple Scab Leaf) with 1938 samples
🧪 Augmenting class 14 (Raspberry leaf) with 1946 samples
🧪 Augmenting class 24 (Tomato leaf yellow virus) with 1946 samples
🧪 Augmenting class 12 (Potato leaf late blight) with 1946 samples
🧪 Augmenting class 22 (Tomato leaf late blight) with 1946 samples
🧪 Augmenting class 4 (Bell_pepper leaf) with 1946 samples
🧪 Augmenting class 25 (Tomato leaf) with 1946 samples
🧪 Augmenting class 7 (Corn Gray leaf spot) with 1946 samples
🧪 Augmenting class 5 (Blueberry leaf) with 1946 samples
🧪 Augmenting class 3 (Bell_pepper leaf spot) with 1946 samples
🧪 Augmenting class 1 (Apple leaf) with 1946 samples
🧪 Augmenting class 18 (Strawberry leaf) with 1946 samples
🧪 Augmenting class 20 (Tomato Septoria leaf spot) with 1946 samples
🧪 Augmenting class 9 (Corn rust leaf) with 1946 samples
🧪 Augmenting class 23 (Tomato leaf mosaic virus) with 1946 samples
🧪 Augmenting class 8 (Corn leaf blight) with 1946 samples
🧪 Augmenting class 2 (Apple rust lea

In [11]:
import os
import shutil
from glob import glob

CLASS_ID = 5  # Grape leaf
NEW_DATA_DIR = "/content/PlantDec_grape_only"
os.makedirs(NEW_DATA_DIR, exist_ok=True)

# Copy base structure
for split in ['images', 'labels']:
    for sub in ['train', 'val']:
        os.makedirs(f"{NEW_DATA_DIR}/{split}/{sub}", exist_ok=True)

def filter_grape_leaf(src_img_dir, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    for lbl_path in glob(f"{src_lbl_dir}/*.txt"):
        with open(lbl_path, "r") as f:
            lines = f.readlines()
        # Keep only labels that match class 5
        filtered_lines = [line for line in lines if int(line.split()[0]) == CLASS_ID]
        if filtered_lines:
            base = os.path.basename(lbl_path).replace(".txt", "")
            img_path = os.path.join(src_img_dir, base + ".jpg")
            if os.path.exists(img_path):
                # Save label and image
                with open(os.path.join(dst_lbl_dir, base + ".txt"), "w") as f:
                    f.writelines(filtered_lines)
                shutil.copy(img_path, os.path.join(dst_img_dir, base + ".jpg"))

# Filter both train and val
filter_grape_leaf(
    src_img_dir=f"{DATA_DIR}/images/train",
    src_lbl_dir=f"{DATA_DIR}/labels/train",
    dst_img_dir=f"{NEW_DATA_DIR}/images/train",
    dst_lbl_dir=f"{NEW_DATA_DIR}/labels/train"
)
filter_grape_leaf(
    src_img_dir=f"{DATA_DIR}/images/val",
    src_lbl_dir=f"{DATA_DIR}/labels/val",
    dst_img_dir=f"{NEW_DATA_DIR}/images/val",
    dst_lbl_dir=f"{NEW_DATA_DIR}/labels/val"
)


In [12]:
import yaml

grape_data_yaml = {
    'train': f'{NEW_DATA_DIR}/images/train',
    'val': f'{NEW_DATA_DIR}/images/val',
    'nc': 1,
    'names': ['grape leaf']
}

with open(f'{NEW_DATA_DIR}/data.yaml', 'w') as f:
    yaml.dump(grape_data_yaml, f)

print("✅ New data.yaml written for grape leaf only.")


✅ New data.yaml written for grape leaf only.


In [13]:
from ultralytics import YOLO
import time

model = YOLO("yolov8s.yaml")  # If you modified this YAML to use SE in C2f

start = time.time()

model.train(
    data=f"{NEW_DATA_DIR}/data.yaml",
    epochs=10,
    imgsz=416,
    batch=8,
    name="yolov11_grape_only_se",
    augment=True
)

print("✅ Training done. Time taken:", round((time.time() - start) / 60, 2), "minutes")


Ultralytics 8.3.132 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/PlantDec_grape_only/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_grape_only_se, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

train: Scanning /content/PlantDec_grape_only/labels/train... 1948 images, 0 backgrounds, 1948 corrupt: 100%|██████████| 1948/1948 [00:00<00:00, 3424.36it/s]

train: /content/PlantDec_grape_only/images/train/image_0205.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/PlantDec_grape_only/images/train/image_0205_aug_0.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/PlantDec_grape_only/images/train/image_0205_aug_1.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/PlantDec_grape_only/images/train/image_0205_aug_10.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/PlantDec_grape_only/images/train/image_0205_aug_100.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/PlantDec_grape_only/images/train/image_0205_aug_1000.jpg: ignoring corrupt image/label: Label class 5 excee

ValueError: not enough values to unpack (expected 3, got 0)

In [18]:
import glob

label_dir = f"{NEW_DATA_DIR}/labels/train"
for label_file in glob.glob(f"{label_dir}/*.txt"):
    with open(label_file, "r") as f:
        lines = f.readlines()
    with open(label_file, "w") as f:
        for line in lines:
            parts = line.strip().split()
            if parts:
                parts[0] = "0"  # convert class 5 ➝ 0
                f.write(" ".join(parts) + "\n")


In [20]:
train: "/content/PlantDec_grape_only/images/train"
val: "/content/PlantDec_grape_only/images/val"
nc: 1
names: ['grape leaf']


In [23]:
import glob

label_dirs = [
    "/content/PlantDec_grape_only/labels/train",
    "/content/PlantDec_grape_only/labels/val"
]

bad_files = []

for label_dir in label_dirs:
    for label_file in glob.glob(f"{label_dir}/*.txt"):
        with open(label_file, "r") as f:
            lines = f.readlines()

        fixed_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts:
                if int(parts[0]) > 0:
                    bad_files.append(label_file)
                parts[0] = "0"
                fixed_lines.append(" ".join(parts) + "\n")

        with open(label_file, "w") as f:
            f.writelines(fixed_lines)

if bad_files:
    print("Fixed label files that had invalid class indices:")
    for f in sorted(set(bad_files)):
        print(" -", f)
else:
    print("✅ All labels are valid now (only class 0).")


✅ All labels are valid now (only class 0).


In [24]:
!rm -f /content/PlantDec_grape_only/labels/train.cache
!rm -f /content/PlantDec_grape_only/labels/val.cache


In [25]:
model.train(
    data=f"{NEW_DATA_DIR}/data.yaml",
    epochs=10,
    imgsz=416,
    batch=8,
    name="yolov11_grape_only_se"
)


Ultralytics 8.3.132 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/PlantDec_grape_only/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_grape_only_se4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

train: Scanning /content/PlantDec_grape_only/labels/train... 1948 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1948/1948 [00:00<00:00, 2361.61it/s]

train: New cache created: /content/PlantDec_grape_only/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


FileNotFoundError: [34m[1mval: [0mError loading data from /content/PlantDec_grape_only/images/val
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [27]:
import os
import random
import shutil

src_img_dir = "/content/PlantDec_grape_only/images/train"
src_lbl_dir = "/content/PlantDec_grape_only/labels/train"

dst_img_dir = "/content/PlantDec_grape_only/images/val"
dst_lbl_dir = "/content/PlantDec_grape_only/labels/val"

os.makedirs(dst_img_dir, exist_ok=True)
os.makedirs(dst_lbl_dir, exist_ok=True)

# Pick 10% of training data for validation
all_imgs = [f for f in os.listdir(src_img_dir) if f.endswith((".jpg", ".png"))]
val_imgs = random.sample(all_imgs, k=max(10, len(all_imgs) // 10))

for img_name in val_imgs:
    # Move image
    shutil.copy(os.path.join(src_img_dir, img_name), dst_img_dir)
    # Move corresponding label
    label_name = img_name.replace(".jpg", ".txt").replace(".png", ".txt")
    shutil.copy(os.path.join(src_lbl_dir, label_name), dst_lbl_dir)

print(f"✅ Moved {len(val_imgs)} images to validation set.")


✅ Moved 194 images to validation set.


In [28]:
model.train(
    data=f"{NEW_DATA_DIR}/data.yaml",
    epochs=10,
    imgsz=416,
    batch=8,
    name="yolov11_grape_only_se"
)


Ultralytics 8.3.132 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/PlantDec_grape_only/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_grape_only_se5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

train: Scanning /content/PlantDec_grape_only/labels/train.cache... 1948 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1948/1948 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 949.3±591.0 MB/s, size: 83.9 KB)


val: Scanning /content/PlantDec_grape_only/labels/val... 194 images, 0 backgrounds, 0 corrupt: 100%|██████████| 194/194 [00:00<00:00, 821.81it/s]

...

                   all        194        194          1          1      0.995      0.895



10 epochs completed in 0.086 hours.
Optimizer stripped from /content/ultralytics/runs/detect/yolov11_grape_only_se5/weights/last.pt, 22.5MB
Optimizer stripped from /content/ultralytics/runs/detect/yolov11_grape_only_se5/weights/best.pt, 22.5MB

Validating /content/ultralytics/runs/detect/yolov11_grape_only_se5/weights/best.pt...
Ultralytics 8.3.132 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  6.03it/s]


                   all        194        194      0.999          1      0.995      0.926
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to /content/ultralytics/runs/detect/yolov11_grape_only_se5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f17fddc1750>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [32]:
import os
from glob import glob

val_images = glob(f"{DATA_DIR}/images/val/*.jpg")
print(f"Total validation images: {len(val_images)}")
print("Sample file names:", [os.path.basename(f) for f in val_images[:5]])


Total validation images: 347
Sample file names: ['stock_photo_peach_leaf_isolated_on_white_background_198744794_jpg.rf.b055fe221c307a15cd4463cc93b59ee7.jpg', 'depositphotos_45748597_stock_photo_ripe_apple_with_leaf_jpg.rf.0b2186761dfd51db01afa099cff4732c.jpg', 'early_symptoms_of_crabapple_scab_on_prairifire_jpg.rf.cbbfb152b4c74ccffa4dee9749551b50.jpg', '1688_jpg.rf.8695d7d36f28dca9871b9c4248920599.jpg', 'prunus_pensylvanica_le_ahaines_a_jpg.rf.1347d6fec1dccb140ee719edc5f93d38.jpg']
